In [14]:
import altair as alt
import pandas as pd
import altair_viewer
alt.data_transformers.disable_max_rows()
altair_viewer._global_viewer._use_bundled_js = False
alt.data_transformers.enable('data_server')

Measure = ['Quantity','Profit','Discount','Sales']



def plotBar(row,column,f):      #string == Measure
    fil = f            ###sum
    fd = fd2           ###month
    if len(column) == 1 or len(row) == 1:     #column 1 , row 1
        if type(row) == type('string'):               #Measure in Row
            sy = str(fil+'('+row+')')
        else:
            if df[row[0]].dtypes == 'datetime64[ns]':   #Dimension(Datetime) in Row
                sy = str(fd+'('+row[0]+')')
            else:
                sy = str(row[0])                        #normal Dimension in Row

        if type(column) == type('string'):           #Measure in Column
            sx = str(fil+'('+column+')')
        else:
            if df[column[0]].dtypes == 'datetime64[ns]':    #Dimension(Datetime) in column
                sx = str(fd+'('+column[0]+')')
            else:
                sx = str(column[0])                     #normal Dimension in Column
        
        c = alt.Chart(df).mark_bar().encode(
            x=sx,
            y=sy,
            tooltip = [sy,sx]
        ).resolve_scale(x = 'independent',y = 'independent')
        return c
    else:
                                           #2 Dimension
        if type(row) == type('string') :               #2 Column (Dimension)
            Di = column
            Me = row
            if df[Di[-2]].dtypes == 'datetime64[ns]':
                scol = str(fd+'('+Di[-2]+')')           
                sx = str(Di[-1])
            elif df[Di[-1]].dtypes == 'datetime64[ns]':
                scol = str(Di[-2])
                sx = str(fd+'('+Di[-1]+')')
            else:
                scol = str(Di[-2])                      #column > x
                sx = str(Di[-1])

            c = alt.Chart(df).mark_bar().encode(
                x=sx,
                y=str(fil+'('+Me+')'),
                #color=scol,
                tooltip = [sx,str(fil+'('+Me+')')]
            ).facet(column=scol
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        elif type(column) == type('string'):                 #2 Row (Dimension)
            Di = row    
            Me = column
            if df[Di[-2]].dtypes == 'datetime64[ns]':            #year,date error (large data)
                srow = str(fd+'('+Di[-2]+')')
                sy = str(Di[-1])

            elif df[Di[-1]].dtypes == 'datetime64[ns]':
                srow = str(Di[-2])
                sy = str(fd+'('+Di[-1]+')')
            else:
                srow = str(Di[-2])
                sy = str(Di[-1])
            c = alt.Chart(df).mark_bar().encode(
                x=str(fil+'('+Me+')'),
                y=sy,
                #color=srow,
                tooltip = [sy,str(fil+'('+Me+')')]
            ).facet(row=srow
            ).resolve_scale(y = 'independent',x = 'independent')
            return c
        else:
            return #'Pls enter 1 or 2 Dimension'
        
def plotGraph(row,column,fil,tp):

    def checkMeasure(R,C):      #True when row is measure
        for r in R:
            if r in Measure:
                return True
            else:
                return False
    f = fil

    if tp == 'line':
        if checkMeasure(row,column):    #row is measure
            if len(row) == 1:
                return alt.vconcat(plotLine(row[0],column,f))
            elif len(row) == 2:
                return alt.vconcat(plotLine(row[0],column,f),plotLine(row[1],column,f))
            elif len(row) == 3:
                return alt.vconcat(plotLine(row[0],column,f),plotLine(row[1],column,f),plotLine(row[2],column,f))
            elif len(row) == 4:
                return alt.vconcat(plotLine(row[0],column,f),plotLine(row[1],column,f),plotLine(row[2],column,f),plotLine(row[3],column,f))
        else:                       #column is Measurement
            if len(column) == 1:
                return plotLine(row,column[0],f)
            elif len(column) == 2:
                return alt.hconcat(plotLine(row,column[0],f),plotLine(row,column[1],f))
            elif len(column) == 3:
                return alt.hconcat(plotLine(row,column[0],f),plotLine(row,column[1],f),plotLine(row,column[2],f))
            elif len(column) == 4:
                return alt.hconcat(plotLine(row,column[0],f),plotLine(row,column[1],f),plotLine(row,column[2],f),plotLine(row,column[3],f))
    
    
    elif tp == 'bar':
        if checkMeasure(row,column):    #row is measure
            if len(row) == 1:
                return alt.vconcat(plotBar(row[0],column,f))
            elif len(row) == 2:
                return alt.vconcat(plotBar(row[0],column,f),plotBar(row[1],column,f))
            elif len(row) == 3:
                return alt.vconcat(plotBar(row[0],column,f),plotBar(row[1],column,f),plotBar(row[2],column,f))
            elif len(row) == 4:
                return alt.vconcat(plotBar(row[0],column,f),plotBar(row[1],column,f),plotBar(row[2],column,f),plotBar(row[3],column,f))
        else:
            if len(column) == 1:
                return alt.hconcat(plotBar(row,column[0],f))
            elif len(column) == 2:
                return alt.hconcat(plotBar(row,column[0],f),plotBar(row,column[1],f))
            elif len(column) == 3:
                return alt.hconcat(plotBar(row,column[0],f),plotBar(row,column[1],f),plotBar(row,column[2],f))
            elif len(column) == 4:
                return alt.hconcat(plotBar(row,column[0],f),plotBar(row,column[1],f),plotBar(row,column[2],f),plotBar(row,column[3],f))

    elif tp == 'pie':
        return plotPie(row,column,f)

def plotLine(row,column,f):

    fil = f                                        ###sum
    fd = fd2                                        ###month
    if type(column) == type('string'):                 #column is Measure
        if len(row) == 1:
            #if df[row[0]].dtypes == 'datetime64[ns]':
            Di = row[0]
            Me = column
            ch = alt.Chart(df).mark_line(point=True).encode(
                alt.X(str(fil+'('+Me+'):Q')),
                alt.Y(str(fd+'('+Di+'):T')),
                tooltip = [str(fd+'('+Di+'):T'),str(fil+'('+Me+'):Q')]
            )
            return ch
        else:
            Me = column
            ch = alt.Chart(df).mark_line(point=True).encode(
                alt.X(str(fil+'('+Me+'):Q')),
                alt.Y(
                    alt.repeat('layer'),timeUnit = fd2,type='temporal',
                ),
                color = alt.datum(alt.repeat('layer')),
                tooltip = [str(fil+'('+Me+')')]
            ).repeat(layer=[row[0],row[1]]
            ).resolve_legend(
                size='independent'
            )
            return ch
    else:   #row is Measure

        #elif df[c].dtypes == 'datetime64[ns]':
        if len(column) == 1:
            Me = row
            Di = column[0]
            ch = alt.Chart(df).mark_line(point=True).encode(
                alt.X(str(fd+'('+Di+'):T')),
                alt.Y(str(fil+'('+Me+'):Q')),
                tooltip = [str(fd+'('+Di+'):T'),str(fil+'('+Me+'):Q')]
            )
            return ch
        else:
            Me = row
            ch = alt.Chart(df).mark_line(point=True).encode(
                alt.X(
                    alt.repeat('layer'),timeUnit = fd2,type='temporal',
                ),
                alt.Y(str(fil+'('+Me+'):Q')),
                color = alt.datum(alt.repeat('layer')),
                tooltip = [str(fil+'('+Me+')')]
            ).repeat(layer=[column[0],column[1]]
            ).resolve_legend(
                size='independent'
            )
            return ch

def plotPie(row,column,f):

    fil = f                                       ####sum
    fd = fd3                                     ####year
    if row[0] in Measure:
        Mes = row[0]
        Di = column[0]
    elif column[0] in Measure:
        Mes = column[0]
        Di = row[0]

    if df[Di].dtypes == 'datetime64[ns]':
        s = str(fd+'('+Di+'):T')
    else:
        s = str(Di+':N')

    base = alt.Chart(df).mark_arc().encode(
        theta=alt.Theta(str(fil+'('+Mes+'):Q')), 
        color=alt.Color(s, type="nominal"), 
        tooltip = [s,str(fil+'('+Mes+'):Q')]
    )
    return base

fd3 = 'year'
fd2 = 'month'

df = pd.read_csv('Superstore.csv', encoding='windows-1252')
df['Order Date'] = pd.to_datetime(df['Order Date'],format='%d/%m/%Y')
df['Ship Date'] = pd.to_datetime(df['Ship Date'],format='%d/%m/%Y')


In [15]:
row = ['Order Date']
column = ['Quantity','Profit','Discount','Sales']
#column , row = row ,column
plotGraph(row,column,'average','bar')

alt.HConcatChart(...)

In [16]:
row = ['Category','Sub-Category']
column = ['Quantity','Profit','Discount','Sales']
#column , row = row ,column
plotGraph(row,column,'sum','bar')

alt.HConcatChart(...)

In [19]:
row = ['Order Date','Ship Date']
column = ['Profit','Sales']
column , row = row ,column
plotGraph(row,column,'count','line')

alt.VConcatChart(...)

In [18]:
row = ['Region']
column = ['Quantity']
plotGraph(row,column,'sum','pie')

alt.Chart(...)

In [21]:
import altair as alt
import pandas as pd
import altair_viewer
alt.data_transformers.disable_max_rows()
altair_viewer._global_viewer._use_bundled_js = False
alt.data_transformers.enable('data_server')

df = pd.read_csv('Superstore.csv', encoding='windows-1252')

c1 = alt.Chart(df).mark_bar().encode(
    x='Sub-Category',
    y='sum(Quantity)'
).facet(column='Category'
).resolve_scale(x = 'independent',y = 'independent')

c2 = alt.Chart(df).mark_bar().encode(
    x='Sub-Category',
    y='sum(Profit)'
).facet(column='Category'
).resolve_scale(x = 'independent',y = 'independent')

c3 = alt.Chart(df).mark_bar().encode(
    x='Sub-Category',
    y='sum(Discount)'
).facet(column='Category'
).resolve_scale(x = 'independent',y = 'independent')

alt.vconcat(c1,c2,c3)


alt.VConcatChart(...)